In [ ]:
# DETECTION MODEL

In [ ]:
import torch
import os
import torchvision
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T
import torch.nn.functional as F

In [ ]:
# CHECKING DATASET STRUCTURE

train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/eval"
test_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"

train_folder=os.listdir(train_path)
eval_folder=os.listdir(eval_path)
test_folder=os.listdir(test_path)

print(f"Nmumber of training samples: {len(train_folder)}")
print(f"Nmumber of evaluation samples: {len(eval_folder)}")
print(f"Nmumber of testing samples: {len(test_folder)}")
print(f"Total number of samples: {len(train_folder)+len(test_folder)+len(eval_folder)}")

# CHECKING ERRORS FOR FUNCTION: get_bounding_box

for file in train_folder: # executed also for eval and test set with no Error raised
    numbers=file.split("-")
    values=numbers[3]

    # if len(values) != 31:                                                                   questo controllo non serve in quanto dipende da quante cifre ogni numero ha
    #     raise ValueError(f" 1 st exception: Invalid plate text format in filename: {file}")
    
    values_v2=values.split("&")

    if len(values_v2) != 5:
        raise ValueError(f"2 nd exception: Invalid plate text format in filename: {file}")
    
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))

    if len(values_v3) != 3:
        raise ValueError(f"3 rd exception: Invalid plate text format in filename: {file}")
    
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]

    if len(x_coords) != 4:
        raise ValueError(f"4 th exception: Invalid plate text format in filename: {file}")

 # CHECKING ERRORS FOR FUNCTION: get_text
    
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

for file in train_folder: # executed also for eval and test set with no Error raised
        values=file.split("-")

        if len(values)!=7:
            raise ValueError(f"1 st exception occurred: Invalid plate text format in filename: {file}")
        
        text=str(values[4])

        # if len(text)!=7:                                                                                questo controllo non serve in quanto dipende da quante cifre ogni numero ha cifre diverse
        #     raise ValueError(f"2 nd exception occurred: Invalid plate text format in filename: {file}") 
        
        indices=text.split("_")

        if len(indices) != 8:
            raise ValueError(f"3 rd exception occurred: Invalid plate text format in filename: {file}")
        
        province_character=provinces[int(indices[0])]
        alphabet_character=alphabet[int(indices[1])]
        ads_charachters=[ads[int(i)] for i in indices[2:]]
        plate_text=province_character+alphabet_character+"".join(ads_charachters)

        if len(plate_text) != 8:
            raise ValueError(f"4 th exception occurred: Invalid plate text format in filename: {file}")

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_bounding_box

file="308069444444444444-91_94-8&424_589&564-578&564_27&553_8&431_589&424-0_0_3_24_32_25_32_25-100-241.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_text

file="30237890625-87_88-197&472_449&567-449&553_201&567_197&472_448&477-10_2_3_24_31_33_26_24-192-75.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file2.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

In [ ]:
# NUOVO DATASET CON PIU ELEMENTI DI TRAIN E MENO DI TEST

path_train_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
path_test_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"
path_train_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/train"
path_test_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test"
folder_train_og=os.listdir(path_train_og)
folder_test_og=os.listdir(path_test_og)
folder_train_tf=os.listdir(path_train_tf)
folder_test_tf=os.listdir(path_test_tf)
print(f"number of original training samples: {len(folder_train_og)}")
print(f"number of original test samples: {len(folder_test_og)}")
print(f"number of new training samples: {len(folder_train_tf)}")
print(f"number of new training samples: {len(folder_test_tf)}")

In [ ]:
# BOUNDING BOX FUNCTION 

def get_bounding_box(file):
    numbers=file.split("-")
    values=numbers[3]
    values_v2=values.split("&")
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)
    
    return [float(x_min), float(y_min), float(x_max), float(y_max)]

In [ ]:
# INTERSECTION OVER UNION FUNCTION

def compute_IoU(box1, box2):

    box1=box1.squeeze()
    box2=box2.squeeze()

    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    area_of_intersection = max(0, xB - xA) * max(0, yB - yA)

    area_box1 = (box1[2] - box1[0]) * (box1[3]- box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    IoU = area_of_intersection / float(area_box1 + area_box2 - area_of_intersection)

    return IoU

In [ ]:
# DATASET   

import os
import torch
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

smaller_train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train-20250508T142228Z-1-001/train"
train_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train2"
eval_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval2"
test_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2"

class CCPD_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        self.folder = os.listdir(path)
        self.images = [f for f in self.folder if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        file = self.images[idx]
        full_path = os.path.join(self.path, file)
        image = Image.open(full_path).convert("RGB")

        bbox = get_bounding_box(file)
        tensor_bbox = torch.tensor([bbox], dtype=torch.float32)
        label = torch.tensor([1], dtype=torch.int64)  

        target = {"boxes": tensor_bbox, "labels": label}

        if target["boxes"].shape[0] != target["labels"].shape[0]:
            raise ValueError(f"Mismatch in number of boxes and labels for file: {file}")

        if self.transforms:
            image = self.transforms(image)

        return image, target
    
def collate_fn(batch):
    return tuple(zip(*batch))

# EXAMPLE OF THE OUTPUT:

# image_path_example="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train/04-90_267-158&448_542&553-541&553_162&551_158&448_542&450-0_1_3_24_27_33_30_24-99-116.jpg"
# images, targets =next(iter(train_dataset))
# print(f"images: {images}, targets: {targets}")



In [ ]:
# CHOICE OF THE DATASET

train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train_medium"
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/eval_medium"
test_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium"

transform = transforms.ToTensor()
train_dataset = CCPD_Dataset(train_path, transforms=transform)
eval_dataset = CCPD_Dataset(eval_path, transforms=transform)
test_dataset = CCPD_Dataset(test_path, transforms=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=0) # watch out for the batch size, it is set to 1 for compatibility with the model
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=0)  # watch out for the batch size, it is set to 1 for compatibility with the model
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=0)  # watch out for the batch size, it is set to 1 for compatibility with the model

In [ ]:
# TRAINING LOOP

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch.optim as optim

model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)
model.rpn.post_nms_top_n_train = 500
model.rpn.post_nms_top_n_test = 500
model.roi_heads.detections_per_img = 50  
# device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device="cpu"
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
best_boxes=[]
best_avg_iou = 0.0
num_epochs=1

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    count=1
    for images, targets in train_dataloader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss += losses.item()
        print(f"Batch: {count} Training Loss: {losses.item():.4f}")
        count+=1

    model.eval()
    total_iou = 0
    count = 0
    with torch.no_grad():
        for images, targets in eval_dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for pred, target in zip(outputs, targets):
                pred_boxes = pred['boxes'].to(device)
                gt_boxes = target['boxes'].to(device)

                if len(pred_boxes) > 0:
                    best_iou = 0.0
                    best_box = None
                    for box, score in zip(pred['boxes'], pred['scores']):
                        if score > 0.0:
                            iou = compute_IoU(box, gt_boxes[0])
                            if iou > best_iou:
                                best_iou = iou
                                best_box = box
                    if best_iou > 0:
                        total_iou += best_iou
                        count += 1
                        best_boxes.append(best_box.cpu())

    avg_iou = total_iou / count if count > 0 else 0

    if avg_iou > best_avg_iou:
        best_avg_iou = avg_iou
        torch.save(model.state_dict(), "best_frcnn_model.pth")
        print(f"Saved best model with IoU: {best_avg_iou:.4f}")

    print(f"Average IoU on eval set: {avg_iou:.4f}")

In [ ]:
# LOADING THE MODEL

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image

def load_Fasterrcnn(device):
    model = fasterrcnn_resnet50_fpn(num_classes=2)  
    model.load_state_dict(torch.load('best_frcnn_model _final_version.pth', map_location="cpu"))
    model.to(device)
    model.eval()
    return model

model=load_Fasterrcnn("cpu")
device="cpu"

In [ ]:
# TESTING THE TRAINED MODEL USING THE BEST BOUNDING BOX

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('best_frcnn_model _final_version.pth'))
device="cpu"
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium")
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        score = output[0]['scores'][0].item()        
        best_bb=pred['boxes'][0].cpu()
        gt_box=get_bounding_box(image_path)
        tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
        iou=compute_IoU(best_bb, tensor_bbox)
        plt.imshow(image)
        ax = plt.gca()
        x1, y1, x2, y2 = best_bb
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1, f" score: {score:.2f}, IoU: {iou:.2f}", color='white', bbox=dict(facecolor='red', alpha=0.5))
        plt.axis('off')
        plt.show()

In [ ]:
# TESTING THE UNTRAINED MODEL USING THE BEST BOUNDING BOX

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
device="cpu"
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2")
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        score = output[0]['scores'][0].item()        
        best_bb=pred['boxes'][0].cpu()
        gt_box=get_bounding_box(image_path)
        tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
        iou=compute_IoU(best_bb, tensor_bbox)
        plt.imshow(image)
        ax = plt.gca()
        x1, y1, x2, y2 = best_bb
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1, f" score: {score:.2f}, IoU: {iou:.2f}", color='white', bbox=dict(facecolor='red', alpha=0.5))
        plt.axis('off')
        plt.show()

In [ ]:
# COMPUTING AVERAGE IOU ON TEST SET USING THE TRAINED MODEL

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('best_frcnn_model _final_version.pth'))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium")
total_iou=0
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        if len(output[0]['scores'])>0:
            score = output[0]['scores'][0].item()        
            best_bb=pred['boxes'][0].to(device)
            gt_box=get_bounding_box(image_path)
            tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
            iou=compute_IoU(best_bb, tensor_bbox)
            total_iou+=iou
print(f"Average IoU: {total_iou/len(folder)}") # 0.87

In [ ]:
# CAR PLATE TEXT FUNCTION

provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

def get_text(file):
    values=file.split("-")
    text=str(values[4])
    indices=text.split("_")
    province_character=provinces[int(indices[0])]
    alphabet_character=alphabet[int(indices[1])]
    ads_charachters=[ads[int(i)] for i in indices[2:]]
    plate_text=province_character+alphabet_character+"".join(ads_charachters)
    return plate_text

In [ ]:
# CROP FUNCTION WITH PREDICTED BOUNDING BOX

def crop_image_with_RCNN(file):
    image = Image.open(file).convert("RGB")
    transform = T.ToTensor()
    img_tensor = transform(image).unsqueeze(0).to(device) 
    with torch.no_grad():
        prediction = model(img_tensor)[0]
        if len(prediction['boxes']) == 0:
            print(f"No box found for image: {file}")
            return None
        best_bb = prediction['boxes'][0].to(device).int()
        cropped = img_tensor[0, :, best_bb[1]:best_bb[3], best_bb[0]:best_bb[2]]
        cropped_resized = F.interpolate(cropped.unsqueeze(0), size=(48, 144), mode='bilinear', align_corners=False)
        return cropped_resized.squeeze(0)  

def crop_folder_with_RCNN(folder_path):
    cropped_folder = []
    files = os.listdir(folder_path)
    for file in files:
        full_path = os.path.join(folder_path, file)
        gt_text=get_text(full_path)
        cropped_image = crop_image_with_RCNN(full_path)
        if cropped_image is not None:
            cropped_folder.append([cropped_image, gt_text])
    return cropped_folder

def crop_image_with_ground_truth(full_path):
    filename = os.path.basename(full_path)  
    bb = get_bounding_box(filename)
    image = Image.open(full_path).convert("RGB")
    transform = T.ToTensor()
    img_tensor = transform(image)
    cropped = img_tensor[:, int(bb[1]):int(bb[3]), int(bb[0]):int(bb[2])]
    cropped_resized = F.interpolate(cropped.unsqueeze(0), size=(48, 144), mode='bilinear', align_corners=False)
    return cropped_resized.squeeze(0)

def crop_folder_with_ground_truth(folder_path):
    cropped_folder = []
    files = os.listdir(folder_path)
    for file in files:
        full_path = os.path.join(folder_path, file)
        gt_text = get_text(file)  # Get ground truth text
        cropped_image = crop_image_with_ground_truth(full_path)
        cropped_folder.append([cropped_image, gt_text])  # Store image and text pair
    return cropped_folder

In [ ]:
# SHOWING THE RESULTS OF THE TRAINED MODEL

for image in os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium"):
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test_medium", image)
        cropped = crop_image_with_RCNN(image_path)
        plt.imshow(cropped.permute(1, 2, 0).cpu().numpy())
        plt.axis("off")
        plt.title("Cropped Image")
        plt.show()

In [14]:
# EXAMPLE OF CROPPING A FOLDER WITH "crop_folder_with_RCNN"

import matplotlib.pyplot as plt
import numpy as np

model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('model_weights/best_frcnn_model.pth'))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval_small"
cropped_folder=crop_folder_with_RCNN(eval_path)

image=cropped_folder[0][0]
image_np = image.permute(1, 2, 0).cpu().numpy()
plt.imshow(image_np)
plt.axis('off')
plt.show()
print(f"ground truth text: {cropped_folder[0][1]}")

/tmp/ipykernel_10227/49714682.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weights/best_frcnn_model.pth'))


KeyboardInterrupt: 

In [ ]:
# SIMPLE PDLPR

import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import mobilenet_v2
from torch.utils.data import Dataset, DataLoader

provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

all_characters = sorted(set(provinces + alphabet + ads))
char_list = sorted(set(provinces + alphabet + ads))
char2idx = {'<blank>': 0}  # blank at index 0
char2idx.update({char: idx + 1 for idx, char in enumerate(char_list)})
idx2char = {idx: char for char, idx in char2idx.items()}


# Dataset
class CroppedImages(Dataset):
    def __init__(self, folder, transformations):
        self.folder = folder
        self.transformations = transformations

    def __len__(self):
        return len(self.folder)

    def __getitem__(self, idx):
        image = self.folder[idx][0]
        gt_text=self.folder[idx][1]
        return self.transformations(image), gt_text

def collate_fn(batch):
    images, labels= zip(*batch)
    images = torch.stack(images)
    return images, labels

class MobilenetV2FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        base = mobilenet_v2(pretrained=True)
        self.backbone = nn.Sequential(*list(base.features.children()))
        self.projection = nn.Sequential(
            nn.Conv2d(1280, 512, kernel_size=1),
            nn.AdaptiveAvgPool2d((6, 18))
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.projection(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=200):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # [1, max_len, d_model]

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class ParallelDecoder(nn.Module):
    def __init__(self, d_model=512, num_chars=8, num_classes=68, num_layers=2):
        super().__init__()
        self.num_chars = num_chars
        self.query_embed = nn.Parameter(torch.randn(num_chars, d_model))

        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=8)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, memory):
        B, _, D = memory.shape
        queries = self.query_embed.unsqueeze(1).repeat(1, B, 1)  
        memory = memory.permute(1, 0, 2)  
        decoded = self.decoder(tgt=queries, memory=memory)  
        decoded = decoded.permute(1, 0, 2)  
        logits = self.classifier(decoded)   
        return logits

trans_train = transforms.Compose([
    transforms.Resize((64, 256)),
    transforms.RandomRotation(degrees=5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trans_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# model.eval()
# cropped_folder_train=crop_folder_with_RCNN("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train2")
# cropped_folder_eval=crop_folder_with_RCNN("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval2")

# train_dataset = CroppedImages(cropped_folder_train, trans_train)
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# eval_dataset = CroppedImages(cropped_folder_eval, trans_eval)
# eval_dataloader = DataLoader(eval_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Instantiate models
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# feature_extractor = MobilenetV2FeatureExtractor().to(device)
# pos_encoder = PositionalEncoding(d_model=512).to(device)

# # Transformer
# encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4).to(device)
# decoder = ParallelDecoder(d_model=512, num_chars=8, num_classes=68).to(device)

def encode_labels(label_list, char2idx, max_len=8):
    encoded = []
    for label in label_list:
        label = label[:max_len].ljust(max_len)
        encoded.append([char2idx[c] for c in label])
    return torch.tensor(encoded, dtype=torch.long)

In [ ]:
# EXAMPLE OF WHAT THE DATASET RETURNS AND CHECK ON ENCODED LABELS

cropped_folder = crop_folder_with_ground_truth("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/single_sample_eval")
dataset = CroppedImages(cropped_folder, trans_eval)
train_dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)
item=next(iter(train_dataloader))
cropped = crop_image_with_ground_truth('/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/single_sample_eval/03625-92_266-225&396_573&501-568&501_227&490_225&396_573&414-0_0_3_17_25_28_27_30-102-83.jpg')
plt.imshow(cropped.permute(1, 2, 0).cpu().numpy())
plt.axis("off")
plt.title("Cropped Image")
plt.show()
print(f"Encoded labels: {encode_labels(item[1],char2idx)}")
predicted_indices= encode_labels(item[1], char2idx).squeeze().tolist()
plate = ''.join([idx2char[i] for i in predicted_indices])
print(f"Decoded labels: {plate}")

In [ ]:
# EXAMPLE OF UNTRAINED PDLPR PREDICTION

with torch.no_grad():
    # Get cropped image and process through model
    cropped_image = crop_image_with_RCNN('/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2/03125-89_263-177&502_477&597-464&581_177&597_185&513_477&502-0_0_3_27_29_25_33_33-102-60.jpg')
    cropped_image = cropped_image.to(device)
    features = feature_extractor(cropped_image.unsqueeze(0).to(device))  
    B, C, H, W = features.shape
    features = features.view(B, C, H * W).permute(0, 2, 1)
    features = pos_encoder(features)
    memory = transformer_encoder(features)
    logits = decoder(memory)
    
    # Get predictions
    predicted_indices = logits.argmax(dim=-1)[0]  # Take first batch item
    predicted_text = ''.join([idx2char[idx.item()] for idx in predicted_indices])
    
    # Get ground truth text from filename
    filename = os.path.basename('/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2/03125-89_263-177&502_477&597-464&581_177&597_185&513_477&502-0_0_3_27_29_25_33_33-102-60.jpg')
    ground_truth = get_text(filename)
    
    # Print results
    print(f"Predicted text: {predicted_text}")
    print(f"Ground truth: {ground_truth}")
    
    # Optionally visualize the cropped image
    plt.figure(figsize=(10, 4))
    plt.imshow(cropped_image.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.show()

In [ ]:
# COMPLETE SIMPLE PDLPR TRAINING AND EVALUATION 

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model components setup 
feature_extractor = MobilenetV2FeatureExtractor().to(device)
pos_encoder = PositionalEncoding(d_model=512).to(device)
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dropout=0.3)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4).to(device)
decoder = ParallelDecoder(d_model=512, num_chars=8, num_classes=68).to(device)

# Create optimizer and scheduler
params = (list(feature_extractor.parameters()) + 
         list(transformer_encoder.parameters()) + 
         list(decoder.parameters()))
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True
)
criterion = nn.CrossEntropyLoss()

# Training parameters
num_epochs = 40
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training loop
for epoch in range(num_epochs):
    # Training phase
    feature_extractor.train()
    transformer_encoder.train()
    decoder.train()
    
    train_loss = 0
    train_correct = 0
    train_total = 0

    for batch_images, batch_labels in train_dataloader:
        batch_images = batch_images.to(device)
        batch_labels_encoded = encode_labels(batch_labels, char2idx).to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        features = feature_extractor(batch_images)
        B, C, H, W = features.shape
        features = features.view(B, C, H * W).permute(0, 2, 1)
        features = pos_encoder(features)
        memory = transformer_encoder(features)
        logits = decoder(memory)

        # Calculate loss
        loss = 0
        for i in range(8):
            loss += criterion(logits[:, i, :], batch_labels_encoded[:, i])
        loss /= 8

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
        optimizer.step()

        train_loss += loss.item()

        # Calculate accuracy
        pred = logits.argmax(dim=-1)
        train_correct += (pred == batch_labels_encoded).sum().item()
        train_total += batch_labels_encoded.numel()

    avg_train_loss = train_loss / len(train_dataloader)
    train_accuracy = train_correct / train_total

    # Evaluation phase
    feature_extractor.eval()
    transformer_encoder.eval()
    decoder.eval()
    
    val_loss = 0
    val_correct = 0 
    val_total = 0

    with torch.no_grad():
        for batch_images, batch_labels in eval_dataloader:
            batch_images = batch_images.to(device)
            batch_labels_encoded = encode_labels(batch_labels, char2idx).to(device)

            # Forward pass
            features = feature_extractor(batch_images)
            B, C, H, W = features.shape
            features = features.view(B, C, H * W).permute(0, 2, 1)
            features = pos_encoder(features)
            memory = transformer_encoder(features)
            logits = decoder(memory)

            # Calculate loss
            loss = 0
            for i in range(8):
                loss += criterion(logits[:, i, :], batch_labels_encoded[:, i])
            loss /= 8

            val_loss += loss.item()

            # Calculate accuracy
            pred = logits.argmax(dim=-1)
            val_correct += (pred == batch_labels_encoded).sum().item()
            val_total += batch_labels_encoded.numel()

    avg_val_loss = val_loss / len(eval_dataloader)
    val_accuracy = val_correct / val_total

    # Learning rate scheduling
    scheduler.step(avg_val_loss)

    # Save best model and early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save({
            'feature_extractor_state_dict': feature_extractor.state_dict(),
            'pos_encoder_state_dict': pos_encoder.state_dict(),
            'transformer_encoder_state_dict': transformer_encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch': epoch,
            'best_val_loss': best_val_loss
        }, 'best_pdlpr_model.pth')
        print(f"Saved new best model with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 50)

    # Early stopping
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break

In [ ]:
# EXAMPLE OF TRAINED PDLPR PREDICTION

def pdlpr_inference(image_path, device='cpu'):
    """
    Perform inference on a single image
    """
    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((48, 144)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load image and model checkpoint
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Initialize models
    feature_extractor = MobilenetV2FeatureExtractor().to(device)
    pos_encoder = PositionalEncoding(d_model=512).to(device)
    encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dropout=0.3)
    transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4).to(device)
    decoder = ParallelDecoder(d_model=512, num_chars=8, num_classes=68).to(device)
    
    # Load checkpoint
    try:
        checkpoint = torch.load('best_pdlpr_model.pth', map_location=device)
        feature_extractor.load_state_dict(checkpoint['feature_extractor_state_dict'])
        pos_encoder.load_state_dict(checkpoint['pos_encoder_state_dict'])
        transformer_encoder.load_state_dict(checkpoint['transformer_encoder_state_dict'])
        decoder.load_state_dict(checkpoint['decoder_state_dict'])
    except Exception as e:
        print(f"Error loading model: {e}")
        return None
    
    # Set models to eval mode
    feature_extractor.eval()
    transformer_encoder.eval()
    decoder.eval()
    
    # Inference
    with torch.no_grad():
        # Feature extraction
        features = feature_extractor(image_tensor)
        B, C, H, W = features.shape
        features = features.view(B, C, H * W).permute(0, 2, 1)
        
        # Add positional encoding
        features = pos_encoder(features)
        
        # Transform features
        memory = transformer_encoder(features)
        
        # Decode
        logits = decoder(memory)
        
        # Get predictions
        predicted_indices = logits.argmax(dim=-1)[0]  # Take first batch item
        plate_text = ''.join([idx2char[idx.item()] for idx in predicted_indices])
        
        return plate_text

# Test function to process multiple images
def test_multiple_images(image_paths):
    """
    Test PDLPR on multiple images and show results
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for img_path in image_paths:
        # Get ground truth text from filename
        gt_text = get_text(os.path.basename(img_path))
        
        # Get prediction
        pred_text = pdlpr_inference(img_path, device)
        
        # Display results
        img = Image.open(img_path)
        plt.figure(figsize=(10, 4))
        plt.imshow(img)
        plt.title(f"GT: {gt_text}\nPred: {pred_text}")
        plt.axis('off')
        plt.show()
        print(f"Image: {os.path.basename(img_path)}")
        print(f"Ground truth: {gt_text}")
        print(f"Prediction: {pred_text}")
        print("-" * 50)

# Example usage

test_images = [
    "/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2/04-91_254-145&472_529&567-529&567_164&552_145&472_529&485-0_0_3_25_24_24_24_29-148-355.jpg",
    "/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2/05-90_257-137&507_572&612-567&611_158&612_137&507_572&512-0_0_3_28_32_25_24_32-144-135.jpg",
    "/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2/035-93_261-170&516_506&611-502&611_177&594_170&516_506&536-0_0_3_28_25_33_32_32-99-116.jpg"
]

test_multiple_images(test_images)

In [ ]:
# PDLPR MODEL FOLLOWING PAPER ARCHITECTURE

import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from torch.nn import CTCLoss

# --- Focus Structure Module ---
class Focus(nn.Module):
    def __init__(self, in_channels=3, out_channels=64, dropout=0.1):
        super(Focus, self).__init__()
        self.conv = nn.Conv2d(in_channels * 4, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(inplace=True)
        self.dropout = nn.Dropout(dropout)
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Slice and concat
        patch1 = x[..., ::2, ::2]
        patch2 = x[..., ::2, 1::2]
        patch3 = x[..., 1::2, ::2]
        patch4 = x[..., 1::2, 1::2]
        x = torch.cat([patch1, patch2, patch3, patch4], dim=1)
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.dropout(x)
        return x


# --- CNN Block used in RESBLOCK and downsampling ---
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(inplace=True)
    )


# --- Residual Block ---
class ResBlock(nn.Module):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.block = nn.Sequential(
            conv_block(channels, channels),
            conv_block(channels, channels)
        )

    def forward(self, x):
        return x + self.block(x)


# --- IGFE Module ---
class IGFE(nn.Module):
    def __init__(self, dropout=0.1):
        super(IGFE, self).__init__()
        self.focus = Focus(3, 64, dropout)
        self.down1 = conv_block(64, 128, stride=2)
        self.res1 = ResBlock(128)
        self.res2 = ResBlock(128)
        self.down2 = conv_block(128, 256, stride=2)
        self.res3 = ResBlock(256)
        self.res4 = ResBlock(256)
        self.final_conv = nn.Conv2d(256, 512, kernel_size=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 18))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.focus(x)
        x = self.down1(x)
        x = self.dropout(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.down2(x)
        x = self.dropout(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.final_conv(x)
        x = self.adaptive_pool(x)
        return x


# --- Positional Encoding for 2D feature maps ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, height, width):
        super().__init__()
        self.pe = nn.Parameter(torch.randn(1, d_model, height, width))

    def forward(self, x):
        return x + self.pe


# --- Transformer Encoder Block ---
class EncoderBlock(nn.Module):
    def __init__(self, d_model):
        super(EncoderBlock, self).__init__()
        self.cnn1 = nn.Conv2d(d_model, d_model * 2, kernel_size=1)
        self.attn = nn.MultiheadAttention(d_model * 2, num_heads=8, batch_first=True)
        self.cnn2 = nn.Conv2d(d_model * 2, d_model, kernel_size=1)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.cnn1(x)  # [B, 1024, H, W]
        x_ = x.flatten(2).transpose(1, 2)  # [B, HW, 1024]
        x_, _ = self.attn(x_, x_, x_)
        x_ = x_.transpose(1, 2).view(B, -1, H, W)  # [B, 1024, H, W]
        x = self.cnn2(x_)
        return self.norm(x.flatten(2).transpose(1, 2)).transpose(1, 2).view(B, C, H, W)

# --- Parallel Decoder Block ---
class DecoderBlock(nn.Module):
    def __init__(self, d_model):
        super(DecoderBlock, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, num_heads=8, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(d_model, num_heads=8, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.ReLU(),
            nn.Linear(d_model * 2, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, tgt, memory):
        # Masked self-attention (causal)
        B, T, _ = tgt.shape
        mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(tgt.device)
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=mask)
        tgt = self.norm1(tgt + tgt2)

        # Cross-attention
        tgt2, _ = self.cross_attn(tgt, memory, memory)
        tgt = self.norm2(tgt + tgt2)

        # Feedforward
        tgt2 = self.ffn(tgt)
        tgt = self.norm3(tgt + tgt2)
        return tgt


# --- Full PDLPR Recognition Model ---
class PDLPR(nn.Module):
    def __init__(self, num_classes, dropout=0.1):
        super(PDLPR, self).__init__()
        self.igfe = IGFE(dropout)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 18))
        self.pos_encoding = PositionalEncoding(512, 6, 18)
        self.encoder = nn.Sequential(*[EncoderBlock(512) for _ in range(3)])
        self.flatten = lambda x: x.flatten(2).transpose(1, 2)
        self.decoder_blocks = nn.ModuleList([DecoderBlock(512) for _ in range(3)])
        self.cls_head = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, tgt_seq):
        # Clean up any GPU memory
        torch.cuda.empty_cache()
        
        x = self.igfe(x)                     
        x = self.adaptive_pool(x)            
        x = self.dropout(x)
        x = self.pos_encoding(x)
        x = self.encoder(x)
        memory = self.flatten(x)             

        tgt = tgt_seq                        
        for block in self.decoder_blocks:
            tgt = block(tgt, memory)
            tgt = self.dropout(tgt)

        logits = self.cls_head(tgt)
        return logits

In [ ]:
# FULL SETUP

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset

def load_Fasterrcnn(device):
    model = fasterrcnn_resnet50_fpn(num_classes=2)  
    model.load_state_dict(torch.load('model_weights/best_frcnn_model.pth'))
    model.to(device)
    model.eval()
    return model

model=load_Fasterrcnn(device="cpu")
device="cpu"
model.eval()

provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

all_characters = sorted(set(provinces + alphabet + ads))
char_list = sorted(set(provinces + alphabet + ads))
char2idx = {'<blank>': 0}  # blank at index 0
char2idx.update({char: idx + 1 for idx, char in enumerate(char_list)})
idx2char = {idx: char for char, idx in char2idx.items()}

# Dataset
class CroppedImages(Dataset):
    def __init__(self, folder, transformations):
        self.folder = folder
        self.transformations = transformations

    def __len__(self):
        return len(self.folder)

    def __getitem__(self, idx):
        try:
            image = self.folder[idx][0]
            gt_text = self.folder[idx][1]
            
            if image is None:
                raise ValueError(f"None image at index {idx}")
                
            if self.transformations:
                image = self.transformations(image)
                
            return image, gt_text
            
        except Exception as e:
            print(f"Error loading sample {idx}: {str(e)}")
            return torch.zeros(3, 48, 144), ""

def encode_labels(label_list, char2idx, max_len=8):
    encoded = []
    for label in label_list:
        label = label[:max_len].ljust(max_len)
        encoded.append([char2idx[c] for c in label])
    return torch.tensor(encoded, dtype=torch.long)

def ctc_collate_fn(batch):
    images, labels = zip(*batch)
    images = torch.stack(images)
    label_lengths = torch.tensor([len(l) for l in labels], dtype=torch.long)
    labels = torch.cat([encode_labels([l], char2idx) for l in labels])
    return images, labels, label_lengths

def ctc_greedy_decoder(output, idx2char, blank=0):
    '''
    Now, I know the network returns probabilities, as it does a softmax with logits of characters.
    I need to transform that probability into an actual char to compose the plate.
    I take the argmax of the softmax (most prob char), remove blanks used by CTC and possible
    duplicates CTC can actually produce.
    At the end I simply use the  mappings char-index index-char deefined at the beginning to compose the plate.
    This is greedy as it just takes the argmax of every step, I think it's more than enough here.
    '''
    # output: [seq_len, batch, num_classes]
    out = output.permute(1, 0, 2)  # [batch, seq_len, num_classes]
    pred_strings = []
    for probs in out:
        pred = probs.argmax(1).cpu().numpy()
        #
        prev = -1
        pred_str = []
        for p in pred:
            if p != blank and p != prev:
                pred_str.append(idx2char[p])
            prev = p
        pred_strings.append(''.join(pred_str))
    return pred_strings

trans_train = transforms.Compose([
    transforms.Resize((48, 144)),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0))
    ], p=0.5),
    transforms.RandomRotation(degrees=(-5, 5)),
    transforms.RandomAffine(
        degrees=0, 
        translate=(0.1, 0.1),
        scale=(0.9, 1.1)
    ),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225]),
])

trans_eval = transforms.Compose([
    transforms.Resize((48, 144)),  # Fixed input size
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# CHOICE OF THE DATASET

model=load_Fasterrcnn(device="cpu")
device="cpu"
model.eval()

cropped_folder_train=crop_folder_with_ground_truth("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train_small")
cropped_folder_eval=crop_folder_with_ground_truth("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval_small")

train_dataset = CroppedImages(cropped_folder_train, trans_train)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=False, collate_fn=ctc_collate_fn)

eval_dataset = CroppedImages(cropped_folder_eval, trans_eval)
eval_dataloader = DataLoader(eval_dataset, batch_size=8, shuffle=False, collate_fn=ctc_collate_fn)


In [ ]:
# COMPLETE PDLPR TRAINING AND EVALUATION WITH CTC LOSS

import torch.optim.lr_scheduler as lr_scheduler
from torch.nn import CTCLoss

# Device setup 
device = torch.device("cpu")

# Model setup
model = PDLPR(num_classes=len(char2idx), dropout=0.1).to(device)
criterion = CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0005)
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min', 
    factor=0.7,
    patience=3,
    verbose=True,
    min_lr=1e-6
)

# CTC Loss initialization

# Training parameters
num_epochs = 15
best_val_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    true_train_accuracy = 0

    for batch_idx, (batch_images, batch_labels, label_lengths) in enumerate(train_dataloader):
        try:
            batch_images = batch_images.to(device)
            batch_labels = batch_labels.to(device)
            label_lengths = label_lengths.to(device)
            
            optimizer.zero_grad()
            
            B = batch_images.size(0)
            tgt_seq = torch.zeros(B, 8, 512).to(device)
            
            # Forward pass 
            logits = model(batch_images, tgt_seq)
            log_probs = logits.log_softmax(2)
            
            input_lengths = torch.full(size=(B,), 
                                    fill_value=8,
                                    dtype=torch.long,
                                    device=device)
            
            # Calculate CTC loss
            loss = criterion(log_probs.permute(1, 0, 2),
                           batch_labels,
                           input_lengths,
                           label_lengths)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()

            # Calculate accuracy using CTC decoder
            pred_text = ctc_greedy_decoder(log_probs.permute(1, 0, 2), idx2char)
            for i in range(B):
                true_text = ''.join([idx2char[idx.item()] for idx in batch_labels[i][:label_lengths[i]]])
                if pred_text[i] == true_text:
                    true_train_accuracy += 1

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}: Loss = {loss.item():.4f}")

        except Exception as e:
            print(f"Error in batch {batch_idx}: {str(e)}")
            continue

    avg_train_loss = train_loss / len(train_dataloader)
    true_train_acc = true_train_accuracy / (len(train_dataloader) * B)

    # Evaluation phase
    model.eval()
    val_loss = 0
    true_val_accuracy = 0

    with torch.no_grad():
        for batch_images, batch_labels, label_lengths in eval_dataloader:
            batch_images = batch_images.to(device)
            batch_labels = batch_labels.to(device)
            label_lengths = label_lengths.to(device)

            B = batch_images.size(0)
            tgt_seq = torch.zeros(B, 8, 512).to(device)

            logits = model(batch_images, tgt_seq)
            log_probs = logits.log_softmax(2)

            input_lengths = torch.full(size=(B,), 
                                    fill_value=8,
                                    dtype=torch.long,
                                    device=device)

            loss = criterion(log_probs.permute(1, 0, 2),
                           batch_labels,
                           input_lengths,
                           label_lengths)

            val_loss += loss.item()

            # Calculate accuracy using CTC decoder
            pred_text = ctc_greedy_decoder(log_probs.permute(1, 0, 2), idx2char)
            for i in range(B):
                true_text = ''.join([idx2char[idx.item()] for idx in batch_labels[i][:label_lengths[i]]])
                if pred_text[i] == true_text:
                    true_val_accuracy += 1

    avg_val_loss = val_loss / len(eval_dataloader)
    true_val_acc = true_val_accuracy / (len(eval_dataloader) * B)

    # Learning rate scheduling
    scheduler.step(avg_val_loss)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch': epoch,
            'best_val_loss': best_val_loss
        }, 'best_pdlpr_model_ctc.pth')
        print(f"Saved new best model with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1

    # Print epoch statistics
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f} | True Train Accuracy: {true_train_acc:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | True Val Accuracy: {true_val_acc:.4f}")
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 50)

    # Early stopping
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break